# Simplified Object Interactions

This notebook demonstrates the **recommended approach** for most users and geologists to interact with Evo geoscience objects.

We use the **typed objects API** (`PointSet`, `Regular3DGrid`, etc.) which provides:
- Simple, intuitive Python classes
- Automatic bounding box calculation
- Rich HTML display with Portal/Viewer links
- Type safety and IDE autocompletion

## Requirements

You must have a Seequent account with the Evo entitlement. You'll need:
- The client ID of your Evo application
- The callback/redirect URL of your Evo application

To obtain these credentials, refer to the [Apps and tokens guide](https://developer.seequent.com/docs/guides/getting-started/apps-and-tokens).

## 1. Authentication

First, we authenticate using the `ServiceManagerWidget`. This handles OAuth login and workspace selection.

In [ ]:
from evo.notebooks import ServiceManagerWidget

# Replace with your Evo app credentials
client_id = "<your-client-id>"
redirect_url = "<your-redirect-url>"

manager = await ServiceManagerWidget.with_auth_code(
    client_id=client_id,
    redirect_url=redirect_url,
    cache_location="./notebook-data",
).login()

## 2. Load the Widgets Extension

The `evo.widgets` extension enables rich HTML rendering of Evo objects in Jupyter notebooks. After loading, objects will automatically display with formatted tables and clickable links.

In [ ]:
%load_ext evo.widgets

## 3. Load Sample Data

We'll load drill hole assay data from a CSV file. The data contains X, Y, Z coordinates along with assay values.

In [ ]:
import pandas as pd

# Load the sample assay data
input_file = "sample-data/WP_assay.csv"
df = pd.read_csv(input_file)

print(f"Loaded {len(df)} points")
df.head()

## 4. Create a PointSet Object

The `PointSetData` class defines the data for a new pointset. The DataFrame must have `x`, `y`, `z` columns for coordinates - any additional columns become point attributes.

Note: We rename the CSV columns to lowercase `x`, `y`, `z` as required by the typed objects API.

In [ ]:
from evo.objects.typed import PointSet, PointSetData

# Prepare the DataFrame with required column names
locations_df = df.rename(columns={"X": "x", "Y": "y", "Z": "z"})

# Create the pointset data
pointset_data = PointSetData(
    name="WP Assay Data",
    description="Drill hole assay data loaded from CSV using the simplified typed objects API",
    locations=locations_df,
)

# Create the pointset in Evo
pointset = await PointSet.create(manager, pointset_data)
print(f"Created pointset with {pointset.num_points} points")

## 5. View the Created Object

Simply displaying the object shows a rich HTML summary with:
- Object metadata (name, type, ID)
- Bounding box information
- Clickable links to view in Evo Portal and Viewer

In [ ]:
# Display the pointset with rich HTML formatting
pointset

## 6. Inspect Object Properties

You can access individual properties programmatically:

In [ ]:
print(f"Name: {pointset.name}")
print(f"Description: {pointset.description}")
print(f"Number of points: {pointset.num_points}")
print(f"Bounding Box: {pointset.bounding_box}")
print(f"Object URL: {pointset.metadata.url}")

## 7. View Available Attributes

The `attributes` property shows all point attributes with their types and statistics:

In [ ]:
pointset.attributes

## 8. Get Data as DataFrame

The `to_dataframe()` method returns all point data including coordinates and attributes:

In [ ]:
# Get the full dataframe
data = await pointset.to_dataframe()
data.head(10)

## 9. Download an Existing Object

To download an existing object, use `object_from_reference` with the object URL or `PointSet.from_reference` for a specific type:

In [ ]:
from evo.objects.typed import object_from_reference

# Download the same object we just created (using its URL)
downloaded = await object_from_reference(manager, pointset.metadata.url)

# Display it - the rich HTML formatter shows the same formatted output
downloaded

## 10. View Objects in Evo Viewer

Use the `get_viewer_url_for_objects` function to generate a URL that opens multiple objects together in the Evo Viewer:

In [ ]:
from evo.widgets import get_viewer_url_for_objects

# Generate a viewer URL for the pointset
viewer_url = get_viewer_url_for_objects(manager, [pointset])
print(f"View in Evo Viewer: {viewer_url}")

## Summary

This notebook demonstrated the simplified workflow for working with Evo geoscience objects:

1. **Authenticate** using `ServiceManagerWidget`
2. **Load the widgets extension** with `%load_ext evo.widgets`
3. **Create objects** using typed data classes like `PointSetData`
4. **View objects** with automatic rich HTML formatting
5. **Access data** via `to_dataframe()` and property accessors
6. **Download objects** using `object_from_reference`

For more advanced typed object examples (grids, masked grids, tensor grids), see the `typed-objects.ipynb` notebook in the `evo-objects` package documentation.